In [3]:
import os
import re
from collections import defaultdict
import math
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess(text):
    if not text:
        return []
    text = text.lower()
    text = text.replace('_', '')
    text = re.sub(r'[^\w\s]', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    porter = PorterStemmer()
    stemmed_tokens = [porter.stem(word) for word in tokens]
    return stemmed_tokens

def read_files(folder):
    data = []
    file_ids = []
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            file_id = filename.split('.')[0]  # 檔案名的數字部分
            file_path = os.path.join(folder, filename)
            with open(file_path, 'r') as f:
                content = f.read()
                processed_tokens = preprocess(content)
                data.append(processed_tokens)
                file_ids.append(file_id)
    return data, file_ids

def calculate_df(documents):
    doc_count = defaultdict(int)
    for tokens in documents:
        unique_terms = set(tokens)
        for term in unique_terms:
            doc_count[term] += 1
    return doc_count

def calculate_tfidf(documents):
    doc_count = calculate_df(documents)
    total_documents = len(documents)
    
    tfidf_vectors = {}
    for doc_id, tokens in documents.items():
        tf = defaultdict(int)
        for token in tokens:
            tf[token] += 1
        
        total_terms = len(tokens)
        for term in tf:
            tf[term] /= total_terms
        
        tfidf = {}
        for term, term_tf in tf.items():
            idf = math.log10(total_documents / (1 + doc_count[term]))
            tfidf[term] = term_tf * idf
        
        norm = math.sqrt(sum(value ** 2 for value in tfidf.values()))
        tfidf_unit_vector = {term: value / norm for term, value in tfidf.items()} if norm > 0 else tfidf
        
        tfidf_vectors[doc_id] = tfidf_unit_vector
    
    return tfidf_vectors

def cosine_similarity(vec1, vec2):
    dot_product = sum(vec1.get(term, 0) * vec2.get(term, 0) for term in set(vec1) | set(vec2))
    magnitude_1 = math.sqrt(sum(value ** 2 for value in vec1.values()))
    magnitude_2 = math.sqrt(sum(value ** 2 for value in vec2.values()))
    if magnitude_1 == 0 or magnitude_2 == 0:
        return 0
    return dot_product / (magnitude_1 * magnitude_2)

def calculate_distance_matrix(tfidf_vectors):
    doc_ids = list(tfidf_vectors.keys())
    distance_matrix = {}
    for i, doc_id_1 in enumerate(doc_ids):
        for j, doc_id_2 in enumerate(doc_ids):
            if i < j:
                sim = cosine_similarity(tfidf_vectors[doc_id_1], tfidf_vectors[doc_id_2])
                distance_matrix[(doc_id_1, doc_id_2)] = 1 - sim
                distance_matrix[(doc_id_2, doc_id_1)] = 1 - sim
    return distance_matrix, doc_ids

def hac_clustering(distance_matrix, doc_ids, k):
    clusters = {doc_id: [doc_id] for doc_id in doc_ids}
    while len(clusters) > k:
        min_dist = float('inf')
        closest_pair = None

        for (doc_id_1, doc_id_2), dist in distance_matrix.items():
            if dist < min_dist:
                min_dist = dist
                closest_pair = (doc_id_1, doc_id_2)
        
        doc_id_1, doc_id_2 = closest_pair
        cluster_1 = clusters.pop(doc_id_1, None)
        cluster_2 = clusters.pop(doc_id_2, None)
        if cluster_1 is None or cluster_2 is None:
            continue
        
        new_cluster_id = f"{doc_id_1}_{doc_id_2}"
        clusters[new_cluster_id] = cluster_1 + cluster_2
        
        keys_to_delete = [(doc_id_1, key) for key in clusters.keys()] + \
                         [(doc_id_2, key) for key in clusters.keys()]
        for key in keys_to_delete:
            if key in distance_matrix:
                del distance_matrix[key]
        
        for other_id in clusters.keys():
            dist_1 = min(distance_matrix.get((doc_id_1, other_id), float('inf')), 
                         distance_matrix.get((other_id, doc_id_1), float('inf')))
            dist_2 = min(distance_matrix.get((doc_id_2, other_id), float('inf')), 
                         distance_matrix.get((other_id, doc_id_2), float('inf')))
            distance_matrix[(new_cluster_id, other_id)] = min(dist_1, dist_2)
            distance_matrix[(other_id, new_cluster_id)] = min(dist_1, dist_2)

    return clusters

def save_results(clusters, file_ids, output_file):
    with open(output_file, 'w') as f:
        for cluster_id, cluster in clusters.items():
            for doc_id in cluster:
                f.write(f"{doc_id} {cluster_id}\n")
    print(f"結果已保存至 {output_file}")

folder = "./data"
data, file_ids = read_files(folder)

documents = {file_ids[i]: data[i] for i in range(len(data))}
tfidf_vectors = calculate_tfidf(documents)

distance_matrix, doc_ids = calculate_distance_matrix(tfidf_vectors)

k_values = [8, 13, 20]
for k in k_values:
    print(f"正在處理 k={k} 的聚類...")
    clusters = hac_clustering(distance_matrix, doc_ids, k)
    output_file = f"clusters_k_{k}.txt"
    save_results(clusters, file_ids, output_file)
    print(f"k={k} 的聚類結果已保存至 {output_file}")


正在處理 k=8 的聚類...


KeyboardInterrupt: 